# HW4 Trees and Forests

Official instructions:

https://www.cs.tufts.edu/cs/135/2023f/hw4.html

This is the *starter code* notebook for Problems 1 and 2, looking at sentiment classification using bag-of-words features of product reviews.

In [2]:
import numpy as np
import pandas as pd
import os
import sys
import time


In [3]:
import sklearn.tree
import sklearn.linear_model
import sklearn.metrics
import sklearn.ensemble
import sklearn.model_selection


In [4]:
# From the HW4 starter code
from pretty_print_sklearn_tree import pretty_print_sklearn_tree


In [5]:
# Plotting utils
import matplotlib
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8') # pretty matplotlib plots

import seaborn as sns
sns.set('notebook', font_scale=1.25, style='whitegrid')


# Load all data from train/valid/test

In [6]:
DATA_DIR = os.path.join("data_product_reviews/")


### Load training

In [7]:
x_tr_df = pd.read_csv(os.path.join(DATA_DIR, 'x_train.csv.zip'))
y_tr_df = pd.read_csv(os.path.join(DATA_DIR, 'y_train.csv'))
x_tr_NF = np.minimum(x_tr_df.values, 1.0).copy()
y_tr_N = y_tr_df.values[:,0].copy()

print("Training data")
print("x_tr_NF.shape: %s" % str(x_tr_NF.shape))
print("y_tr_N.shape : %s" % str(y_tr_N.shape))
print("mean(y_tr_N) : %.3f" % np.mean(y_tr_N))


Training data
x_tr_NF.shape: (6346, 7729)
y_tr_N.shape : (6346,)
mean(y_tr_N) : 0.500


### Load validation set

In [8]:
x_va_df = pd.read_csv(os.path.join(DATA_DIR, 'x_valid.csv.zip'))
y_va_df = pd.read_csv(os.path.join(DATA_DIR, 'y_valid.csv'))

x_va_TF = np.minimum(x_va_df.values, 1.0).copy()
y_va_T = y_va_df.values[:,0].copy()

print("Validation data")
print("x_va_TF.shape: %s" % str(x_va_TF.shape))
print("y_va_T.shape : %s" % str(y_va_T.shape))
print("mean(y_va_T) : %.3f" % np.mean(y_va_T))


Validation data
x_va_TF.shape: (792, 7729)
y_va_T.shape : (792,)
mean(y_va_T) : 0.490


### Load test set 

In [9]:
x_te_df = pd.read_csv(os.path.join(DATA_DIR, 'x_test.csv.zip'))
y_te_df = pd.read_csv(os.path.join(DATA_DIR, 'y_test.csv'))

x_te_TF = np.minimum(x_te_df.values, 1.0).copy()
y_te_T = y_te_df.values[:,0].copy()

print("Heldout Test data")
print("x_te_TF.shape: %s" % str(x_te_TF.shape))
print("y_te_T.shape : %s" % str(y_te_T.shape))
print("mean(y_te_T) : %.3f" % np.mean(y_te_T))


Heldout Test data
x_te_TF.shape: (793, 7729)
y_te_T.shape : (793,)
mean(y_te_T) : 0.515


### Load vocabulary as a list of strings

In [10]:
vocab_list = x_tr_df.columns.tolist()


In [11]:
for word in vocab_list[:8]:
    print(word)
print("...")
for word in vocab_list[-8:]:
    print(word)


good
great
time
book
don't
work
i_have
read
...
never_get
i'd_like
loves_it
an_author
nomin
could_give
bad_but
gap


### Pack training and validation sets into big arrays (so we can use sklearn's hyperparameter search tools)

In [12]:
xall_LF = np.vstack([x_tr_NF, x_va_TF])
yall_L = np.hstack([y_tr_N, y_va_T])


In [13]:
valid_indicators_L = np.hstack([
    -1 * np.ones(y_tr_N.size), # -1 means never include this example in any test split
    0  * np.ones(y_va_T.size), #  0 means include in the first test split (we count starting at 0 in python)
    ])


In [14]:
# Create splitter object using Predefined Split
# Will be used later by all hyperparameter searches
my_splitter = sklearn.model_selection.PredefinedSplit(valid_indicators_L)


# Problem 1: Decision Trees

## 1A: Train a simple tree with depth 3

In [15]:
simple_tree = sklearn.tree.DecisionTreeClassifier(
    criterion='gini', 
    max_depth=3, min_samples_split=2, min_samples_leaf=1, 
    random_state=101)


### **Fit the tree** 

**TODO Train on the training set** in the next coding cell

In [16]:
simple_tree.fit(x_tr_NF, y_tr_N)


DecisionTreeClassifier(max_depth=3, random_state=101)

### **Figure 1: Print Tree** 

Use a helper function from the starter code

In [17]:
# call pretty_print_sklearn_tree on simple_tree, providing feature_names=vocab_list
pretty_print_sklearn_tree(simple_tree, feature_names=vocab_list)


The binary tree structure has 15 nodes.
- depth   0 has    1 nodes, of which    0 are leaves
- depth   1 has    2 nodes, of which    0 are leaves
- depth   2 has    4 nodes, of which    0 are leaves
- depth   3 has    8 nodes, of which    8 are leaves
The decision tree:  (Note: Y = 'yes' to above question; N = 'no')
Decision: X['great'] <= 0.50?
  Y Decision: X['excel'] <= 0.50?
    Y Decision: X['disappoint'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.430 (4041 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.114 (368 total training examples)
    N Decision: X['disappoint'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.903 (277 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.429 (14 total training examples)
  N Decision: X['return'] <= 0.50?
    Y Decision: X['bad'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.745 (1413 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.415 (142 total training examples)
    N Decision: X['movie'] <= 0.50?
    

## 1B : Find best Decision Tree with grid search

In [18]:
# Construct the default predictor
# Any hyperparameters here may be overridden by the hyperparameter grid

base_tree = sklearn.tree.DecisionTreeClassifier(
    criterion='gini',
    min_samples_split=2, min_samples_leaf=1)


In [19]:
tree_hyperparameter_grid_by_name = dict(
    max_depth=[2, 8, 32, 128],
    min_samples_leaf=[1, 3, 9],
    random_state = [101],
    )


**TODO Build the Grid Search** in the next coding cell

Follow the 1B instructions carefully

In [20]:
tree_grid_searcher = sklearn.model_selection.GridSearchCV(estimator=base_tree, param_grid=tree_hyperparameter_grid_by_name, scoring="balanced_accuracy", cv=my_splitter, return_train_score=True, refit=False, verbose=3, n_jobs=-1)


### Do the search!


In [21]:
start_time_sec = time.time()
tree_grid_searcher.fit(xall_LF, yall_L)
elapsed_time_sec = time.time() - start_time_sec


Fitting 1 folds for each of 12 candidates, totalling 12 fits


[CV 1/1] END max_depth=2, min_samples_leaf=1, random_state=101;, score=(train=0.640, test=0.634) total time=   3.0s
[CV 1/1] END max_depth=2, min_samples_leaf=9, random_state=101;, score=(train=0.640, test=0.634) total time=   3.0s
[CV 1/1] END max_depth=2, min_samples_leaf=3, random_state=101;, score=(train=0.640, test=0.634) total time=   3.0s
[CV 1/1] END max_depth=8, min_samples_leaf=3, random_state=101;, score=(train=0.723, test=0.698) total time=   5.4s
[CV 1/1] END max_depth=8, min_samples_leaf=9, random_state=101;, score=(train=0.712, test=0.696) total time=   5.6s
[CV 1/1] END max_depth=8, min_samples_leaf=1, random_state=101;, score=(train=0.728, test=0.699) total time=   5.6s
[CV 1/1] END max_depth=32, min_samples_leaf=9, random_state=101;, score=(train=0.812, test=0.720) total time=   9.4s
[CV 1/1] END max_depth=32, min_samples_leaf=3, random_state=101;, score=(train=0.875, test=0.735) total time=   9.6s
[CV 1/1] END max_depth=32, min_samples_leaf=1, random_state=101;, scor

### Build dataframe of results

Move the results of grid search into a nice pandas data frame.

In [22]:
if hasattr(tree_grid_searcher, 'cv_results_'):
    # Will be here if you called fit on tree_grid_searcher succesfully
    tree_search_results_df = pd.DataFrame(tree_grid_searcher.cv_results_).copy()
    print("Grid search of %3d configurations done after %6.1f sec" % (
        tree_search_results_df.shape[0], elapsed_time_sec))


Grid search of  12 configurations done after   13.6 sec


### Display search results

This block will make a pretty printed table of the results of your grid search

In [23]:
if hasattr(tree_grid_searcher, 'cv_results_'):
    pd.set_option('display.precision', 4)
    tree_keys = ['param_max_depth', 'param_min_samples_leaf']
    tree_search_results_df.sort_values(tree_keys, inplace=True)
    tree_search_results_df[tree_keys + ['mean_train_score', 'mean_test_score', 'rank_test_score', 'mean_fit_time']]


In [24]:
print("Printing a dict of the best hyperparameters")
try:
    print(tree_grid_searcher.best_params_)
except AttributeError:
    print("TODO")


Printing a dict of the best hyperparameters
{'max_depth': 128, 'min_samples_leaf': 1, 'random_state': 101}


### Build the best decision tree

**TODO Build the Best Tree on the training set** in the next coding cell

This is necessary so you have the specific best performing tree in your workspace.

Although you fit many trees in the search, they were not stored, so we need to recreate the best one.

Hint: Just feed the best hyperparameters as keyword args to construct the tree and fit it to the training data.

In [25]:
best_tree = base_tree.set_params(**tree_grid_searcher.best_params_)
best_tree.fit(x_tr_NF, y_tr_N)


DecisionTreeClassifier(max_depth=128, random_state=101)

### Interpret the best decision tree

In [26]:
pretty_print_sklearn_tree(best_tree, feature_names=vocab_list)


The binary tree structure has 1587 nodes.
- depth   0 has    1 nodes, of which    0 are leaves
- depth   1 has    2 nodes, of which    0 are leaves
- depth   2 has    4 nodes, of which    0 are leaves
- depth   3 has    8 nodes, of which    0 are leaves
- depth   4 has   16 nodes, of which    5 are leaves
- depth   5 has   22 nodes, of which    6 are leaves
- depth   6 has   32 nodes, of which   13 are leaves
- depth   7 has   38 nodes, of which   21 are leaves
- depth   8 has   34 nodes, of which   16 are leaves
- depth   9 has   36 nodes, of which   13 are leaves
- depth  10 has   46 nodes, of which   23 are leaves
- depth  11 has   46 nodes, of which   26 are leaves
- depth  12 has   40 nodes, of which   17 are leaves
- depth  13 has   46 nodes, of which   25 are leaves
- depth  14 has   42 nodes, of which   24 are leaves
- depth  15 has   36 nodes, of which   18 are leaves
- depth  16 has   36 nodes, of which   16 are leaves
- depth  17 has   40 nodes, of which   22 are leaves
- de

# Problem 2: Random forest

## 2A: Train a random forest with default settings

In [27]:
simple_forest = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_features='sqrt',
    max_depth=3,
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=101)


### Fit the forest

**TODO Train on the training set** in the next coding cell

In [28]:
simple_forest.fit(x_tr_NF, y_tr_N)


RandomForestClassifier(max_depth=3, random_state=101)

## 2B & Table 2: Feature Importances

### Table 2
**Sample Output** (Feel free to print all words and organize them in any software)

|**Important Words**|**Unimportant Words**|
|:-:|:-:|
|I1 |  U1  |
|I2 |  U2  |
|I3 |  U3  |
|I4 |  U4  |
|I5 |  U5  |
|I6 |  U6  |
|I7 |  U7  |
|I8 |  U8  |
|I9 |  U9  |
|I0 |  U0  |

In [102]:
vocab = np.array(vocab_list)
highest_idx = np.argsort(simple_forest.feature_importances_)[::-1]

nonzero_idx = np.flatnonzero(simple_forest.feature_importances_)
nonzero = simple_forest.feature_importances_[nonzero_idx]
lowest_idx =  nonzero_idx[np.argsort(nonzero)]

candidate_size = len(lowest_idx)
highest = highest_idx[:10]
lowest = lowest_idx[:10]

print("Most Important")
for i in highest:
    print(f"{vocab[i]:>10} : {simple_forest.feature_importances_[i]}")

print(f"\nLeast Important (selected from {candidate_size} candidates)")
for i in lowest:
    print(f"{vocab[i]:>10} : {simple_forest.feature_importances_[i]}")

# for i, j in zip(highest, lowest):
#     print(f"{vocab[i]} | {vocab[j]}")


Most Important
    return : 0.03352193274341395
     excel : 0.029508752406818176
     great : 0.029312077860780873
     worst : 0.028433172248247457
      poor : 0.02687213992403518
disappoint : 0.024367869364236888
    i_love : 0.018491552236597205
your_money : 0.018001848624119115
     don't : 0.01755354610600821
      bore : 0.01639521208784837

Least Important (selected from 448 candidates)
     agree : 1.0915292670708816e-07
unless_you : 5.020022697466931e-07
     month : 6.479008296181788e-07
   full_of : 7.046779975180687e-07
     but_i : 4.99054716366578e-06
    make_a : 6.165744894788045e-06
    comedi : 6.724783190596581e-06
      hand : 1.1121502161786002e-05
    for_my : 1.1239170070574924e-05
  you_want : 1.6704822160323164e-05


## 2C: Best Random Forest via grid search

Follow the instructions and using what you learn in 1B to finish this step.

This block might take 2-10 minutes. (Takes about 2 min on staff Macbook laptops.)

If yours runs significantly longer, try this out on Google Colab instead.

In [63]:
base_forest = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_depth=16,
    min_samples_split=2,
    min_samples_leaf=1,
)


In [64]:
forest_hyperparameter_grid_by_name = dict(
    max_features=[3, 10, 33, 100, 333],
    max_depth=[16, 32],
    min_samples_leaf=[1],
    n_estimators=[100],
    random_state=[101],
)


In [75]:
forest_searcher = sklearn.model_selection.GridSearchCV(
    estimator=base_forest, 
    param_grid=forest_hyperparameter_grid_by_name, 
    return_train_score=True, 
    verbose=3,
    n_jobs=-1,
)


### Do the search!

In [90]:
forest_searcher.fit(xall_LF, yall_L)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 3/5] END max_depth=16, max_features=3, min_samples_leaf=1, n_estimators=100, random_state=101;, score=(train=0.922, test=0.765) total time=   0.8s
[CV 5/5] END max_depth=16, max_features=3, min_samples_leaf=1, n_estimators=100, random_state=101;, score=(train=0.918, test=0.754) total time=   0.8s
[CV 1/5] END max_depth=16, max_features=3, min_samples_leaf=1, n_estimators=100, random_state=101;, score=(train=0.920, test=0.762) total time=   0.9s
[CV 2/5] END max_depth=16, max_features=3, min_samples_leaf=1, n_estimators=100, random_state=101;, score=(train=0.924, test=0.781) total time=   0.9s
[CV 4/5] END max_depth=16, max_features=3, min_samples_leaf=1, n_estimators=100, random_state=101;, score=(train=0.920, test=0.761) total time=   0.9s
[CV 3/5] END max_depth=16, max_features=10, min_samples_leaf=1, n_estimators=100, random_state=101;, score=(train=0.936, test=0.817) total time=   1.2s
[CV 2/5] END max_depth=16, max_f

GridSearchCV(estimator=RandomForestClassifier(max_depth=16), n_jobs=-1,
             param_grid={'max_depth': [16, 32],
                         'max_features': [3, 10, 33, 100, 333],
                         'min_samples_leaf': [1], 'n_estimators': [100],
                         'random_state': [101]},
             return_train_score=True, verbose=3)

### Build dataframe of results

In [91]:
pd.DataFrame(forest_searcher.cv_results_)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_features,param_min_samples_leaf,param_n_estimators,param_random_state,params,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.7841,0.0162,0.0706,0.0082,16,3,1,100,101,"{'max_depth': 16, 'max_features': 3, 'min_samp...",...,0.7645,0.0089,10,0.9198,0.9242,0.9215,0.9196,0.9179,0.9206,0.0021
1,1.1808,0.0450,0.0748,0.0086,16,10,1,100,101,"{'max_depth': 16, 'max_features': 10, 'min_sam...",...,0.8176,0.0076,6,0.9366,0.9303,0.9364,0.9352,0.9322,0.9342,0.0025
2,2.4985,0.0236,0.0616,0.0141,16,33,1,100,101,"{'max_depth': 16, 'max_features': 33, 'min_sam...",...,0.8261,0.0081,4,0.9377,0.9313,0.9305,0.9249,0.9265,0.9302,0.0045
3,6.3169,0.0798,0.0593,0.0074,16,100,1,100,101,"{'max_depth': 16, 'max_features': 100, 'min_sa...",...,0.8225,0.0082,5,0.9301,0.9200,0.9208,0.9230,0.9240,0.9236,0.0036
4,18.0868,0.0908,0.0618,0.0084,16,333,1,100,101,"{'max_depth': 16, 'max_features': 333, 'min_sa...",...,0.7910,0.0087,8,0.8741,0.8788,0.8771,0.8745,0.8836,0.8776,0.0035
5,0.9000,0.0231,0.1007,0.0084,32,3,1,100,101,"{'max_depth': 32, 'max_features': 3, 'min_samp...",...,0.7854,0.0128,9,0.9811,0.9783,0.9760,0.9779,0.9772,0.9781,0.0017
6,1.7516,0.0208,0.0919,0.0096,32,10,1,100,101,"{'max_depth': 32, 'max_features': 10, 'min_sam...",...,0.8341,0.0079,2,0.9750,0.9753,0.9715,0.9723,0.9736,0.9735,0.0015
7,4.1734,0.0434,0.1003,0.0120,32,33,1,100,101,"{'max_depth': 32, 'max_features': 33, 'min_sam...",...,0.8345,0.0044,1,0.9737,0.9713,0.9685,0.9702,0.9692,0.9706,0.0018
8,10.1474,0.0630,0.0910,0.0127,32,100,1,100,101,"{'max_depth': 32, 'max_features': 100, 'min_sa...",...,0.8281,0.0047,3,0.9722,0.9715,0.9706,0.9743,0.9743,0.9725,0.0015
9,19.0439,0.0726,0.0554,0.0048,32,333,1,100,101,"{'max_depth': 32, 'max_features': 333, 'min_sa...",...,0.8092,0.0094,7,0.9581,0.9595,0.9557,0.9520,0.9550,0.9561,0.0026


### Display search results

In [93]:
print(f"Best max_features is  {forest_searcher.best_params_['max_features']}")
print(f"Maximum possible is {forest_searcher.n_features_in_}")


Best max_features is  33
Maximum possible is 7729


### Build the best random forest using the best hyperparameters found in 2B 

This is necessary so you have the specific best performing forest in your workspace.

Train *only* on training set (do not merge train and valid)


In [105]:
best_forest = sklearn.ensemble.RandomForestClassifier()
best_forest.set_params(**forest_searcher.best_params_)
best_forest.fit(x_tr_NF, y_tr_N)


RandomForestClassifier(max_depth=32, max_features=33, random_state=101)

### Table 3: Comparison of methods on the bag-of-words to sentiment classification task.

Please report **balanced accuracy** on the train, valid, and test sets, to 3 digits of precision

**Sample Output** (Feel free to print all values and organize them by hand)

|**method**|**max depth**|**num trees**|**train BAcc**|**valid BAcc**|**test BAcc**|
|:-|:-:|:-:|:-:|:-:|:-:|
|simple Tree	| 1 | 1 | 0.123	|0.456	|0.890|
|best Tree	|1 | 1 | 0.123	|0.456	|0.890|
|simple RandomForest	|1 | 1 | 0.123	|0.456	|0.890|
|best RandomForest	|1 | 1 | 0.123	|0.456	|0.890|

In [106]:
classifiers = [simple_tree, best_tree, simple_forest, best_forest]

for c in classifiers:
    yhat_tr_N = c.predict(x_tr_NF)
    yhat_va_T = c.predict(x_va_TF)
    yhat_te_T = c.predict(x_te_TF)
    score_tr = sklearn.metrics.balanced_accuracy_score(y_tr_N, yhat_tr_N)
    score_va = sklearn.metrics.balanced_accuracy_score(y_va_T, yhat_va_T)
    score_te = sklearn.metrics.balanced_accuracy_score(y_te_T, yhat_te_T)
    print(f"{score_tr:.3} | {score_va:.3} | {score_te:.3}")


0.646 | 0.645 | 0.646
0.998 | 0.737 | 0.726
0.816 | 0.797 | 0.773
0.969 | 0.837 | 0.834
